In [1]:
import json

train_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\train.json'
dev_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\dev.json'
test_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\test-unlabelled.json'


def get_content_from_json(file_path):
    document_list = []
    label_list = []
    negative_document = []
    neg_label = []
    with open(file_path) as json_file:
        data = json.load(json_file)   
        for item in data:
            text = data[item]['text'].encode('ascii','ignore').decode("utf-8")  #Encode Decode \u
            document_list.append(text) 
            label_list.append(data[item]['label'])
    return document_list, label_list

pos_data,pos_label= get_content_from_json(train_path)
dev_data,dev_label = get_content_from_json(dev_path)



100

In [2]:
# test data
import json


test_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\test-unlabelled.json'


def get_content_from_json(file_path):
    document_list = []
    label_list = []
    with open(file_path) as json_file:
        data = json.load(json_file)   
        for item in data:
            text = data[item]['text'].encode('ascii','ignore').decode("utf-8")  #Encode Decode \u
            document_list.append(text) 
          
        
    return document_list


test_data = get_content_from_json(test_path)



In [2]:
import pandas as pd


neg_data = pd.read_csv('C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\old_new_ds_climate_new_exp.csv')

pd.set_option('use_inf_as_na', True)
neg_dataset = neg_data.dropna().reset_index(drop=True)

neg_data=neg_dataset['newdescp'].values.tolist()

neg_label = [0] * len(neg_data)

len(neg_label)

1636

In [3]:
train_data = pos_data + neg_data
train_label = pos_label + neg_label

In [4]:
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet



lemmatizer = WordNetLemmatizer()
porter_stemmer = PorterStemmer()
stopwords = set(stopwords.words('English'))

table = str.maketrans('', '', string.punctuation)

def pos_tag(word):
    pos_tag = nltk.tag.pos_tag([word])   
    tag = pos_tag[0][1][0]
    pos_tag_dict = { 'N' : wordnet.NOUN, 'J' : wordnet.ADJ , 'V' : wordnet.VERB, 'R' : wordnet.ADV}
    return pos_tag_dict.get(tag,'n')


def document_preprocessing(data):
    
    token_clean_corpus= []
    #tokenize
    token_corpus =  [word_tokenize(doc) for doc in data]

    #bow representation
    for doc in token_corpus:
        #remove punctuation
        strip_token = [word.translate(table) for word in doc]
        #handle casing
        lower_token = [word.lower() for word in strip_token]
        #remove numbers 
        only_words = [word for word in lower_token if word.isalpha()]
        #remove stopwords
        token_no_stopword = [word for word in only_words if word not in stopwords]
        #stem words
        stem_token = [porter_stemmer.stem(word) for word in token_no_stopword]
        #lemmatize not good
        #lemma_token = [lemmatizer.lemmatize(word,pos_tag(word)) for word in token_no_stopword]
        
       # token_dict = dict(Counter(stem_token))
        token_clean_corpus.append(stem_token)
        
    return token_clean_corpus
    
train_preprocessed_data = document_preprocessing(train_data)
dev_preprocessed_data = document_preprocessing(dev_data) 
#test_preprocessed_data = document_preprocessing(test_data) 
len(train_preprocessed_data),len(dev_preprocessed_data)#,len(test_preprocessed)


(2804, 100)

In [5]:
train_sent =  [' '.join(i) for i in train_preprocessed_data]
dev_sent =  [' '.join(i) for i in dev_preprocessed_data]
#test_sent =  [' '.join(i) for i in test_preprocessed_data]


In [6]:
def bigram_calc(vocab):
#bigram_model.vocab
    bigram_model = Phrases(vocab, min_count=10)
    return bigram_model

In [7]:
import gensim
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.models import Phrases

stopwords = set(stopwords.words('English'))
def preprocess_topic_model(data):
    remove_new_line = [doc.replace('\n','') for doc in data ]
    preprocess_token = [simple_preprocess(doc, deacc=True) for doc in remove_new_line] 
    token_no_stopword = [[word for word in doc if word not in stopwords] for doc in preprocess_token]
    bigram_model = bigram_calc(token_no_stopword)
    bigram_tokens = [bigram_model[doc] for doc in token_no_stopword]
    id2word = gensim.corpora.Dictionary(bigram_tokens)
    id2word.filter_extremes(no_below=10, no_above=0.25);id2word.compactify()
    doc_corpus = [id2word.doc2bow(doc) for doc in bigram_tokens] 

    return doc_corpus, id2word

preprocess_pos_corpus,id2pos_word=preprocess_topic_model(pos_data)
preprocess_neg_corpus,id2neg_word=preprocess_topic_model(neg_data)
preprocess_dev_corpus,id2dev_word=preprocess_topic_model(dev_data)

unable to import 'smart_open.gcs', disabling that module


In [45]:
len(preprocess_dev_corpus)

100

In [ ]:
from gensim.models import HdpModel

def get_topics(doc_corpus,id2word):
    hdp = HdpModel(doc_corpus, id2word=id2word)
    topics = hdp.show_topics(num_topics=500)
    return hdp,topics

pos_hdp,pos_topics=get_topics(preprocess_pos_corpus,id2pos_word)
#neg_hdp,neg_topics=get_topics(preprocess_neg_corpus,id2neg_word)
dev_hdp,dev_topics=get_topics(preprocess_dev_corpus,id2dev_word)

In [181]:
import pandas as pd

def topic_prob_extractor(gensim_hdp):
    shown_topics = gensim_hdp.show_topics(num_topics=150, formatted=False)
    topics_nos = [x[0] for x in shown_topics ]
    weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]

    return pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

In [277]:
df = topic_prob_extractor(dev_hdp)


In [8]:
import logging
from logging import warnings

def  lda_model(doc_corpus,id2word):
    
    logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=doc_corpus,
                           num_topics=20,
                           id2word=id2word,
                           chunksize=150,
                           workers=7, 
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    return lda_train
pos_lda=lda_model(preprocess_pos_corpus,id2pos_word)
neg_lda=lda_model(preprocess_neg_corpus,id2neg_word)
dev_lda=lda_model(preprocess_dev_corpus,id2dev_word)

In [8]:
import re
import matplotlib.pyplot as plt
p = re.compile("(-*\d+\.\d+) per-word .* (\d+\.\d+) perplexity")
matches = [p.findall(l) for l in open('lda_model.log')]
matches = [m for m in matches if len(m) > 0]
tuples = [t[0] for t in matches]
perplexity = [float(t[1]) for t in tuples]
liklihood = [float(t[0]) for t in tuples]
iter = list(range(0,len(tuples)*10,10))
plt.plot(iter,liklihood,c="black")
plt.ylabel("log liklihood")
plt.xlabel("iteration")
plt.title("Topic Model Convergence")
plt.grid()
#plt.savefig("convergence_liklihood.pdf")
plt.show()

<Figure size 640x480 with 1 Axes>

In [28]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(neg_lda,preprocess_neg_corpus,id2neg_word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.159930  0.096459       1        1  12.098234
1      0.066790 -0.088343       2        1  11.878745
18    -0.030513 -0.109652       3        1  11.184752
8      0.134735 -0.057070       4        1   7.396620
3      0.098723 -0.103202       5        1   6.387266
2     -0.171276 -0.088591       6        1   5.946731
0     -0.242383  0.090841       7        1   5.150960
9      0.026662 -0.003693       8        1   5.019997
5      0.083851 -0.075543       9        1   4.839684
6     -0.140301 -0.068243      10        1   4.421672
7      0.086996  0.217904      11        1   4.299999
19    -0.170349 -0.016461      12        1   4.289823
4      0.156010 -0.035523      13        1   3.662859
12    -0.069228 -0.123673      14        1   2.926552
17     0.121659  0.022341      15        1   2.910716
13     0.154949  0.143313      16        1   2.257822
10    -0.130263 -0.024210      17        1   1.971165
15    -0.168285  0.146999      18        1   1.685086
11    -0.115496  0.116587      19        1   1.073553
16     0.147791 -0.040238      20        1   0.597762, topic_info=                Term         Freq        Total Category  logprob  loglift
1981  climate_change  2282.000000  2282.000000  Default  30.0000  30.0000
28         australia  1478.000000  1478.000000  Default  29.0000  29.0000
1634          energy  1110.000000  1110.000000  Default  28.0000  28.0000
1980         climate  2109.000000  2109.000000  Default  27.0000  27.0000
1996           fires   348.000000   348.000000  Default  26.0000  26.0000
...              ...          ...          ...      ...      ...      ...
1119           using    15.754635   371.137756  Topic20  -5.1493   1.9603
1188        increase    14.476221   423.266632  Topic20  -5.2339   1.7442
768        countries    15.127353   540.847412  Topic20  -5.1899   1.5431
818             high    14.294916   445.622498  Topic20  -5.2465   1.6802
1980         climate    14.167087  2109.478027  Topic20  -5.2555   0.1165

[1355 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
3334      3  0.988938         aap_cnn
2         3  0.989720       abc_teams
1900      2  0.062072           abuse
1900      3  0.093108           abuse
1900      7  0.031036           abuse
...     ...       ...             ...
5480      6  0.990691           yemen
5974      6  0.909684          yemeni
5974     10  0.041349          yemeni
5127     14  0.935588  young_hopefuls
7071      6  0.934684          youths

[5815 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 2, 19, 9, 4, 3, 1, 10, 6, 7, 8, 20, 5, 13, 18, 14, 11, 16, 12, 17])

In [9]:
def get_topic_features(data,lda_train,doc_corpus):
    
    train_topics = []
    for i in range(len(data)):
        top_topics = lda_train.get_document_topics(doc_corpus[i], minimum_probability=0.0)
        topics = [top_topics[i][1] for i in range(20)]
        train_topics.append(topics)
    return train_topics

pos_features = get_topic_features(pos_data,pos_lda,preprocess_pos_corpus)
dev_features = get_topic_features(dev_data,dev_lda,preprocess_dev_corpus)
neg_features = get_topic_features(neg_data,neg_lda,preprocess_neg_corpus)

In [10]:
train_features = pos_features+neg_features
len(dev_features),len(dev_data),len(preprocess_dev_corpus)

(100, 100, 100)

In [52]:
from sklearn.preprocessing import StandardScaler


scalar = StandardScaler()
train_scale=scalar.fit_transform(train_features)
dev_scale=scalar.transform(dev_features)

In [54]:
lr = LogisticRegression()
lr.fit(train_scale,train_label)
pred = lr.predict(dev_scale)
score = f1_score(dev_label, pred)
print('F1 Score: %.3f' % score)

F1 Score: 0.449


In [11]:
from sklearn.base import BaseEstimator, TransformerMixin


class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    
class TextStats(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return data_dict[self.key]

In [56]:
train_sent =  [' '.join(i) for i in train_preprocessed_data]
dev_sent =  [' '.join(i) for i in dev_preprocessed_data]
#test_sent =  [' '.join(i) for i in test_preprocessed_data]


In [12]:
data = {'text': [text for text in train_sent],
        'topic': [topic for topic in train_features]}


In [13]:
dev_data = {'text': [text for text in dev_sent],
        'topic': [topic for topic in dev_features]}

In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import RandomForestClassifier,StackingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

estimator_clfs = [('svc',SVC(C=100)), ('mnb',MultinomialNB()),
             ('rf',RandomForestClassifier(n_estimators=200))
             ]

pipeline = Pipeline([
    # Use FeatureUnion to combine the features from topics
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from topics
            ('topic', Pipeline([
                ('selector', ItemSelector(key='topic')),
                ('standard', MinMaxScaler()),
            ])),

            # Pipeline for standard bag-of-words model for body
            ('body_bow', Pipeline([
                ('selector', ItemSelector(key='text')),
                ('tfidf', TfidfVectorizer()),
             
            ])),

            ]
    )),

    # Use a SVC classifier on the combined features
    ('clf',StackingClassifier(estimators=estimator_clfs))
])







pipeline.fit(data,train_label)

Pipeline(memory=None,
         steps=[('union',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('topic',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  ItemSelector(key='topic')),
                                                                 ('standard',
                                                                  MinMaxScaler(copy=True,
                                                                               feature_range=(0,
                                                                                              1)))],
                                                          verbose=False)),
                                                ('body_bow',
                                                 Pipeline(memory=None,
                                   

In [26]:
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score,precision_recall_fscore_support,confusion_matrix,classification_report

pred = pipeline.predict(dev_data)

score = f1_score(dev_label, pred)
print('F1 Score: %.3f' % score)
cm = confusion_matrix(dev_label,pred)
print(cm)
print(classification_report(dev_label,pred))
pre,recall,fscore,_ = precision_recall_fscore_support(dev_label, pred,pos_label=1, average="binary")
print('Avg Precision: %.3f' % pre)
print('Avg recall: %.3f'% recall)
print('Avg fscore: %.3f'% fscore)

F1 Score: 0.727
[[44  6]
 [18 32]]
              precision    recall  f1-score   support

           0       0.71      0.88      0.79        50
           1       0.84      0.64      0.73        50

    accuracy                           0.76       100
   macro avg       0.78      0.76      0.76       100
weighted avg       0.78      0.76      0.76       100

Avg Precision: 0.842
Avg recall: 0.640
Avg fscore: 0.727


In [16]:
from sklearn.metrics import f1_score,classification_report,confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC

estimator_clfs = [('mnb', MultinomialNB()),('sgd',SGDClassifier(loss = 'squared_loss')),('svc',SVC(C=10)),
             ('per',PassiveAggressiveClassifier(C=10))
             ]

clf = StackingClassifier(estimators=estimator_clfs, final_estimator=LogisticRegression(C=100))

clf.fit(train_set,train_label)
pred = clf.predict(develop_set)

score = f1_score(dev_label, pred)
print('F1 Score: %.3f' % score)

F1 Score: 0.813


In [307]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1)
clf.fit(train_scale,train_label)
pred = clf.predict(dev_scale)

score = f1_score(dev_label, pred)
print('F1 Score: %.3f' % score)

F1 Score: 0.621


In [292]:
len(dev_scale)

100

In [26]:
tf_idf_features =tf_idf.get_feature_names()


['aa', 'aa index', 'aa index measur', 'aa index measur sun', 'aa index measur sun magnet', 'aachen', 'aachen univers', 'aachen univers appli', 'aachen univers appli scienc', 'aachen univers appli scienc univers']


In [33]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,learning_method='online',learning_decay=0.9)
lda.fit(pos_set)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.9,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [34]:


def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
#display_topics(lda, tf_idf_features, 10)

print(lda.score(pos_set))
print(lda.perplexity(pos_set))

-1064112.216614969
5066999861268.763


In [37]:
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import f1_score,confusion_matrix,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold


lgb = LGBMClassifier(importance_type='gain')
#parameters = {'learning_rate': (0.0001, 0.001, 0.01,0.07, 0.1, 0.2, 0.3)}
#kfold=KFold(n_splits=10,shuffle=True,random_state=42)
#grid_search = GridSearchCV(lgb, parameters, n_jobs=-1,verbose=1,cv=kfold, scoring='f1')
#grid_search.fit(train_set, train_label)

#print('Best score: %0.3f' % grid_search.best_score_)
#print('Best parameters set:')

#best_parameters = grid_search.best_estimator_.get_params()
#for param_name in sorted(parameters.keys()):
#     print('\t%s: %r' % (param_name, best_parameters[param_name]))

#predictions = grid_search.predict(develop_set)
#score = f1_score(dev_label, predictions)
#print('F1 Score: %.3f' % score)



lgb.fit(train_set,train_label)

pred = lgb.predict(develop_set)

score = f1_score(dev_label, pred)
cm = confusion_matrix(dev_label,pred)
print(cm)
print(classification_report(dev_label,pred))
print('F1 Score: %.3f' % score)
pred

[[29 21]
 [ 5 45]]
              precision    recall  f1-score   support

           0       0.85      0.58      0.69        50
           1       0.68      0.90      0.78        50

    accuracy                           0.74       100
   macro avg       0.77      0.74      0.73       100
weighted avg       0.77      0.74      0.73       100

F1 Score: 0.776


array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1])

In [478]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier



model = XGBClassifier(learning_rate =0.07,n_jobs=-1)

model.fit(train_set,train_label)
#pred = model.predict(develop_set)

#score = f1_score(dev_label, pred)
#print('F1 Score: %.3f' % score)


F1 Score: 0.794


In [43]:
test_pred = sgd.predict(test_set)

pred_dict = {}

for i, val in enumerate(test_pred):
    pred_dict["test-"+str(i)] = {"label": int(val)}

with open('C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\test-output.json', 'w') as outfile:
    json.dump(pred_dict, outfile)